In [1]:
# Import the libraries 
import nltk 
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re
from nltk.stem import WordNetLemmatizer
import numpy as np 
import gensim
from gensim.models import Word2Vec
import pandas as pd 

In [2]:
# Load the dataset 
df_data = pd.read_csv("E:\Tarang\Ashoka\Python\PYTHON PROJECT\PM_Modi_Speech_Text_final.csv")

In [3]:
# Initializing the model 
lemmatizer = WordNetLemmatizer()

In [4]:
def tokenize(speech):
    # Sentence Tokenization
    sentence = nltk.sent_tokenize(speech)

    # Creating an empty list to store the cleaned dataset 
    corpus = []

    # Loop through each sentence in the speech
    for i in range(len(sentence)):
        # Remove any non-alphabetic characters in the sentence and convert to lowercase
        review = re.sub("[^a-zA-Z]", " ", sentence[i])
        review = review.lower()
        
        # Tokenize the sentence into a list of words
        review = review.split()
        
        # Lemmatize each word in the sentence and remove stop words
        lemmatizer = WordNetLemmatizer()
        review = [lemmatizer.lemmatize(word) for word in review if word not in set(stopwords.words("english"))]
        
        # Convert the list of words back into a sentence
        review = ' '.join(review)
        
        # Add the cleaned sentence to the corpus
        corpus.append(review)

    # Tokenize each sentence in the corpus into a list of words
    corpus_tokenized = [nltk.word_tokenize(sentence) for sentence in corpus]

    # Train the Word2Vec model on the tokenized corpus
    model = Word2Vec(sentences=corpus_tokenized, vector_size=300, window=5, min_count=1, workers=4)

    # Extract the word vectors for each word in the corpus
    vectors = []
    for sentence in corpus_tokenized:
        for word in sentence:
            if word in model.wv.key_to_index:
                # If the word is in the Word2Vec model's vocabulary, extract its vector
                word_vector = model.wv.get_vector(word)
                vectors.append(word_vector)
            else:
                # If the word is not in the vocabulary, print a message to the console
                print(f"{word} not in vocabulary.")
    
    # Calculate the average vector for the entire speech
    vectors_array = np.array(vectors)
    avg_vector = np.mean(vectors_array, axis=0)
    
    return avg_vector


In [19]:
# Converts the Speech column to a string data type.
df_data['Speech'] = df_data['Speech'].astype(str)

In [12]:
# Vectorize every speech 
df_data['Vectorized_Speech'] = df_data['Speech'].apply(tokenize)

RuntimeError: you must first build vocabulary before training the model

In [17]:
def tokenize(speech):
    # Sentence Tokenization
    sentence = nltk.sent_tokenize(speech)

    # Creating an empty list to store the cleaned dataset 
    corpus = []

    # Loop through each sentence in the speech
    for i in range(len(sentence)):
        # Remove any non-alphabetic characters in the sentence and convert to lowercase
        review = re.sub("[^a-zA-Z]", " ", sentence[i])
        review = review.lower()
        
        # Tokenize the sentence into a list of words
        review = review.split()
        
        # Lemmatize each word in the sentence and remove stop words
        lemmatizer = WordNetLemmatizer()
        review = [lemmatizer.lemmatize(word) for word in review if word not in set(stopwords.words("english"))]
        
        # Convert the list of words back into a sentence
        review = ' '.join(review)
        
        # Add the cleaned sentence to the corpus
        corpus.append(review)

    # Tokenize each sentence in the corpus into a list of words
    corpus_tokenized = [nltk.word_tokenize(sentence) for sentence in corpus]

    # Build the vocabulary for the Word2Vec model
    model = Word2Vec(vector_size=300, window=5, min_count=1, workers=4)
    model.build_vocab(sentences=corpus_tokenized)

    # Train the Word2Vec model on the tokenized corpus
    #model.train(sentences=corpus_tokenized, total_examples=len(corpus_tokenized), epochs=100)

    # Extract the word vectors for each word in the corpus
    vectors = []
    for sentence in corpus_tokenized:
        for word in sentence:
            if word in model.wv.key_to_index:
                # If the word is in the Word2Vec model's vocabulary, extract its vector
                word_vector = model.wv.get_vector(word)
                vectors.append(word_vector)
            else:
                # If the word is not in the vocabulary, print a message to the console
                print(f"{word} not in vocabulary.")
    
    # Calculate the average vector for the entire speech
    vectors_array = np.array(vectors)
    avg_vector = np.mean(vectors_array, axis=0)
    
    return avg_vector



In [20]:

# Vectorize every speech 
df_data['Vectorized_Speech'] = df_data['Speech'].apply(tokenize)

TypeError: Either one of corpus_file or corpus_iterable value must be provided

In [21]:

def tokenize(speech):
    # Sentence Tokenization
    sentence = nltk.sent_tokenize(speech)

    # Creating an empty list to store the cleaned dataset 
    corpus = []

    # Loop through each sentence in the speech
    for i in range(len(sentence)):
        # Remove any non-alphabetic characters in the sentence and convert to lowercase
        review = re.sub("[^a-zA-Z]", " ", sentence[i])
        review = review.lower()
        
        # Tokenize the sentence into a list of words
        review = review.split()
        
        # Lemmatize each word in the sentence and remove stop words
        lemmatizer = WordNetLemmatizer()
        review = [lemmatizer.lemmatize(word) for word in review if word not in set(stopwords.words("english"))]
        
        # Convert the list of words back into a sentence
        review = ' '.join(review)
        
        # Add the cleaned sentence to the corpus
        corpus.append(review)

    # Tokenize each sentence in the corpus into a list of words
    corpus_tokenized = [nltk.word_tokenize(sentence) for sentence in corpus]

    # Train the Word2Vec model on the tokenized corpus
    model = Word2Vec(corpus_tokenized, vector_size=300, window=5, min_count=1, workers=4)

    # Extract the word vectors for each word in the corpus
    vectors = []
    for sentence in corpus_tokenized:
        for word in sentence:
            if word in model.wv.key_to_index:
                # If the word is in the Word2Vec model's vocabulary, extract its vector
                word_vector = model.wv.get_vector(word)
                vectors.append(word_vector)
            else:
                # If the word is not in the vocabulary, print a message to the console
                print(f"{word} not in vocabulary.")
    
    # Calculate the average vector for the entire speech
    vectors_array = np.array(vectors)
    avg_vector = np.mean(vectors_array, axis=0)
    
    return avg_vector

# Vectorize every speech 
df_data['Vectorized_Speech'] = df_data['Speech'].apply(tokenize)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


RuntimeError: you must first build vocabulary before training the model